In [1]:
import numpy as np
import tsplib95
from python_tsp.distances.tsplib_distance import tsplib_distance_matrix

dataset = "data/xqf131.tsp"

initial_cities = np.atleast_2d(list(tsplib95.load(dataset).node_coords.values()))
distance_matrix = tsplib_distance_matrix(dataset)

In [2]:
def can_stop(CLUSTERS, min_size):
    for c in CLUSTERS:
        if len(c) <= min_size:
            return True
    return False

In [6]:
from algo.get_clusters import get_clusters

CLUSTERS, CENTROIDS = get_clusters(initial_cities, 5)

In [8]:
can_stop(CLUSTERS, 10)

False

In [5]:
from algo.concatenate_all import concatenate_all

result, centroid = concatenate_all(CLUSTERS, CENTROIDS)

In [18]:
from algo.get_clusters import get_clusters
from algo.concatenate_all import concatenate_all

def solve_cluster_tsp(initial_cities, N_CLUSTERS = 8, MIN_SIZE = 10):

    while True:
        CLUSTERS, CENTROIDS = get_clusters(initial_cities, N_CLUSTERS)
        # for idx, c in enumerate(CLUSTERS):
            # print(idx, c)
            
        if can_stop(CLUSTERS, MIN_SIZE):
            return concatenate_all(CLUSTERS, CENTROIDS)
        
        for cluster in CLUSTERS:
            solve_cluster_tsp(cluster, N_CLUSTERS, MIN_SIZE)

In [19]:
result, _ = solve_cluster_tsp(initial_cities)
print(result)

[[ 57  25]
 [ 57  12]
 [ 48   6]
 [ 34   5]
 [ 33  15]
 [ 28  16]
 [ 25  15]
 [ 25  11]
 [ 25   9]
 [ 18  13]
 [ 18  11]
 [ 15   8]
 [ 12   5]
 [  8   0]
 [  2   0]
 [  5   8]
 [  0  13]
 [  5  19]
 [  5  13]
 [  9  10]
 [ 10  10]
 [ 11  10]
 [ 12  10]
 [ 15  13]
 [ 18  15]
 [ 18  17]
 [ 18  19]
 [ 15  19]
 [ 18  21]
 [ 18  23]
 [ 18  25]
 [ 15  25]
 [ 18  27]
 [ 18  33]
 [ 18  35]
 [ 18  37]
 [ 18  39]
 [ 18  41]
 [ 18  42]
 [ 18  44]
 [ 18  45]
 [ 15  43]
 [  5  43]
 [  0  39]
 [  5  37]
 [  0  27]
 [  0  26]
 [  5  25]
 [  5  31]
 [ 15  37]
 [ 15  31]
 [ 18  31]
 [ 18  29]
 [ 25  29]
 [ 28  30]
 [ 28  34]
 [ 28  40]
 [ 28  43]
 [ 28  47]
 [ 34  41]
 [ 34  38]
 [ 38  34]
 [ 41  36]
 [ 41  35]
 [ 41  34]
 [ 41  32]
 [ 38  30]
 [ 35  31]
 [ 34  31]
 [ 34  29]
 [ 34  26]
 [ 33  26]
 [ 32  26]
 [ 33  29]
 [ 33  31]
 [ 32  31]
 [ 28  28]
 [ 25  28]
 [ 25  26]
 [ 25  24]
 [ 25  23]
 [ 25  22]
 [ 28  20]
 [ 34  15]
 [ 35  17]
 [ 38  16]
 [ 38  20]
 [ 40  22]
 [ 41  23]
 [ 48  22]
 [ 48  27]